In [1]:
# tf.estimatorにおける入力関数の作成方法を学ぶ
# 序盤は説明なのでコメントばかり。後半に実際のコードがある。

# input_fnは、特徴量データとターゲットデータをtrain,evaluate,predict関数に渡すことために使われる。
# tf.estimator Quickstart tutorialでは、以下の例になっていた。
# (classifier,trainでinput_fnが使われている)

"""
import numpy as np

training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING, target_dtype=np.int, features_dtype=np.float32)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(training_set.data)},
    y=np.array(training_set.target),
    num_epochs=None,
    shuffle=True)

classifier.train(input_fn=train_input_fn, steps=2000)
"""


'\nimport numpy as np\n\ntraining_set = tf.contrib.learn.datasets.base.load_csv_with_header(\n    filename=IRIS_TRAINING, target_dtype=np.int, features_dtype=np.float32)\n\ntrain_input_fn = tf.estimator.inputs.numpy_input_fn(\n    x={"x": np.array(training_set.data)},\n    y=np.array(training_set.target),\n    num_epochs=None,\n    shuffle=True)\n\nclassifier.train(input_fn=train_input_fn, steps=2000)\n'

In [2]:
# 自作の入力関数を作る場合の基本構成はfeature_colsとlabelsを返すこと。
# fearuter_cols : keyとvalueが結びついた辞書
# labels : label値が格納されているテンソル.正解データと思えば多分Ok.

"""
def my_input_fn():
    # Preprocess your data here...

    # ...then return 1) a mapping of feature columns to Tensors with
    # the corresponding feature data, and 2) a Tensor containing labels
    return feature_cols, labels
"""

'\ndef my_input_fn():\n    # Preprocess your data here...\n\n    # ...then return 1) a mapping of feature columns to Tensors with\n    # the corresponding feature data, and 2) a Tensor containing labels\n    return feature_cols, labels\n'

In [3]:
# numpyやpandasの使用も可能。形式をコンバートしてくれるような関数が用意されている。

"""
import numpy as np
# numpy input_fn.
my_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x_data)},
    y=np.array(y_data),
    ...)
"""

"""
import pandas as pd
# pandas input_fn.
my_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=pd.DataFrame({"x": x_data}),
    y=pd.Series(y_data),
    ...)
"""


'\nimport pandas as pd\n# pandas input_fn.\nmy_input_fn = tf.estimator.inputs.pandas_input_fn(\n    x=pd.DataFrame({"x": x_data}),\n    y=pd.Series(y_data),\n    ...)\n'

In [4]:
# スパースなデータに対しては、SparseTensorを使うと良い。
# 以下の3つを引数に与えれば使える。
# dense_shape: 各次元の要素数
# indices: 非ゼロ値の入っているindex
# values: 1次元のテンソル。indicesに対応した値が入っている。
# 以下のようにSparseTensorを生成できる。denseでの表記も確認のため記載。
# SparseTensorの詳細は以下
# https://www.tensorflow.org/api_docs/python/tf/SparseTensor

"""
sparse_tensor = tf.SparseTensor(indices=[[0,1], [2,4]],
                                values=[6, 0.5],
                                dense_shape=[3, 5])
"""

"""
[[0, 6, 0, 0, 0]
 [0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0.5]]
"""

'\n[[0, 6, 0, 0, 0]\n [0, 0, 0, 0, 0]\n [0, 0, 0, 0, 0.5]]\n'

In [5]:
# 自分のモデルにinput_fn データを渡すには、引数input_fnに与えれば良い
# 注意点としては、input_fnには返り値を与えるのではなく、あくまでも関数オブジェクトを渡すこと。
"""
classifier.train(input_fn=my_input_fn, steps=2000)
"""
"""
# こっちはエラー
classifier.train(input_fn=my_input_fn(training_set), steps=2000)
"""

# もし、入力関数をパラメタライズしたいなら、
# 方法1.ラッパー関数を作ってしまう。
"""
def my_input_fn(data_set):
  ...

def my_input_fn_training_set():
  return my_input_fn(training_set)

classifier.train(input_fn=my_input_fn_training_set, steps=2000)
"""
# 方法2.functools.partialを使って関数オブジェクト化してしまう。
"""
classifier.train(
    input_fn=functools.partial(my_input_fn, data_set=training_set),
    steps=2000)
"""

# 方法3. lambdaを使ってラップする
"""
classifier.train(input_fn=lambda: my_input_fn(training_set), steps=2000)
"""

# この方法(方法3限定?)はtrainだけではなく、evaluate、predictにも使えるのでよく覚えておく。
# (つまり、入力関数をパラメタライズしておくと、方法1のように(?)input_fn_train,input_fn_test,input_fn_predictといった複数の入力関数を用意する必要がなくなる)
"""
classifier.evaluate(input_fn=lambda: my_input_fn(test_set), steps=2000)
"""

'\nclassifier.evaluate(input_fn=lambda: my_input_fn(test_set), steps=2000)\n'

In [6]:
# 入力関数input_fnを作るにあたっては、tf.estimator.inputにある関数が便利。
# num_epochsやshuffleも制御できることも利点。
"""
import pandas as pd

def get_input_fn_from_pandas(data_set, num_epochs=None, shuffle=True):
  return tf.estimator.inputs.pandas_input_fn(
      x=pdDataFrame(...),
      y=pd.Series(...),
      num_epochs=num_epochs,
      shuffle=shuffle)
"""

"""
import numpy as np

def get_input_fn_from_numpy(data_set, num_epochs=None, shuffle=True):
  return tf.estimator.inputs.numpy_input_fn(
      x={...},
      y=np.array(...),
      num_epochs=num_epochs,
      shuffle=shuffle)
"""

'\nimport numpy as np\n\ndef get_input_fn_from_numpy(data_set, num_epochs=None, shuffle=True):\n  return tf.estimator.inputs.numpy_input_fn(\n      x={...},\n      y=np.array(...),\n      num_epochs=num_epochs,\n      shuffle=shuffle)\n'

In [7]:
###############################################
# ここから実践コード
###############################################

In [8]:
# 各種インポートを行い、詳細なログ出力用にlogging vervosityをINFOに設定
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [9]:
# データセットのカラム名を定義し、CSVファイルを読み込む
# また、あとで使うために特徴量で使用するカラム名と
# ラベルで使用するカラム名を分けておく。
COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age",
           "dis", "tax", "ptratio", "medv"]
FEATURES = ["crim", "zn", "indus", "nox", "rm",
            "age", "dis", "tax", "ptratio"]
LABEL = "medv"

training_set = pd.read_csv("boston_train.csv", skipinitialspace=True,
                           skiprows=1, names=COLUMNS)
test_set = pd.read_csv("boston_test.csv", skipinitialspace=True,
                       skiprows=1, names=COLUMNS)
prediction_set = pd.read_csv("boston_predict.csv", skipinitialspace=True,
                             skiprows=1, names=COLUMNS)

In [10]:
# 学習のために、FeatureColumnを作る。
# 連続値を取り扱うのでreal_valued_column()を使う?
feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]

In [11]:
feature_cols

[_NumericColumn(key='crim', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='zn', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='indus', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='nox', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='rm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='dis', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='tax', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='ptratio', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [12]:
# DNN回帰をモデルとしてインスタンス化
# 隠れ層のノード数(2レイヤとも10ノード)を定義し、feature_columnsを与える。
regressor = tf.estimator.DNNRegressor(feature_columns=feature_cols,
                                      hidden_units=[10, 10],
                                      model_dir="/tmp/boston_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_log_step_count_steps': 100, '_model_dir': '/tmp/boston_model', '_tf_random_seed': 1, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None}


In [13]:
# regressor(回帰モデル?)に入力データを渡すための関数を定義
# input_fnを返す関数(wrapper)を定義することでdata_setごとにinput_fnを作れる。
# つまり、training_setに対するinput_fn、test_setに対するinput_fn・・・のように。
# num_epochsについては、繰り返しの回数であって、evaluateやpredictの際には1を指定する。
# それ以外の場合(training)の場合は学習回数?に相当する?
# shuffleについては、学習時にTrue、評価・推定時にはFalse
def get_input_fn(data_set, num_epochs=None, shuffle=True):
  return tf.estimator.inputs.pandas_input_fn(
      x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),
      y = pd.Series(data_set[LABEL].values),
      num_epochs=num_epochs,
      shuffle=shuffle)

In [14]:
# 学習する
regressor.train(input_fn=get_input_fn(training_set), steps=5000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/boston_model\model.ckpt-5000
INFO:tensorflow:Saving checkpoints for 5001 into /tmp/boston_model\model.ckpt.
INFO:tensorflow:step = 5001, loss = 1848.8
INFO:tensorflow:global_step/sec: 500.759
INFO:tensorflow:step = 5101, loss = 4534.5 (0.199 sec)
INFO:tensorflow:global_step/sec: 545.36
INFO:tensorflow:step = 5201, loss = 7335.77 (0.183 sec)
INFO:tensorflow:global_step/sec: 536.416
INFO:tensorflow:step = 5301, loss = 4541.39 (0.186 sec)
INFO:tensorflow:global_step/sec: 552.941
INFO:tensorflow:step = 5401, loss = 4580.51 (0.181 sec)
INFO:tensorflow:global_step/sec: 502.364
INFO:tensorflow:step = 5501, loss = 4986.07 (0.199 sec)
INFO:tensorflow:global_step/sec: 599.959
INFO:tensorflow:step = 5601, loss = 5215.17 (0.167 sec)
INFO:tensorflow:global_step/sec: 517.803
INFO:tensorflow:step = 5701, loss = 2699.61 (0.193 sec)
INFO:tensorflow:global_step/sec: 572.867
INFO:tensorflow:step = 5801, loss = 3716

In [15]:
# テストデータを使って評価する
ev = regressor.evaluate(
    input_fn=get_input_fn(test_set, num_epochs=1, shuffle=False))

INFO:tensorflow:Starting evaluation at 2017-09-10-15:21:08
INFO:tensorflow:Restoring parameters from /tmp/boston_model\model.ckpt-10000
INFO:tensorflow:Finished evaluation at 2017-09-10-15:21:09
INFO:tensorflow:Saving dict for global step 10000: average_loss = 11.4846, global_step = 10000, loss = 1148.46


In [16]:
# 推定する
# predictはiteratorを返すのでlist化してprint
y = regressor.predict(
    input_fn=get_input_fn(prediction_set, num_epochs=1, shuffle=False))
# .predict() returns an iterator of dicts; convert to a list and print
# predictions
predictions = list(p["predictions"] for p in itertools.islice(y, 6))
print("Predictions: {}".format(str(predictions)))

INFO:tensorflow:Restoring parameters from /tmp/boston_model\model.ckpt-10000
Predictions: [array([ 33.71652222], dtype=float32), array([ 17.15636635], dtype=float32), array([ 23.24656677], dtype=float32), array([ 34.67623901], dtype=float32), array([ 14.61829662], dtype=float32), array([ 18.75601768], dtype=float32)]
